In [ ]:
%run setup.py
import numpy as np, pandas as pd, torch
from pathlib import Path
from src.models import default_experts, MoEClassifier

subset = np.load("models/moe_selected_idxs.npy").tolist()

experts = default_experts(
    emb_path="../data/processed/question_embeddings.npy",
    lr_path="models/sbert_lr.pkl"
)
experts = [experts[i] for i in subset]

moe = MoEClassifier(experts, lr=1, epochs=0)
moe.gate.load_state_dict(torch.load("models/moe_gate_state.pt", map_location="cpu"))

test_df = pd.read_csv("../data/splits/test.csv").dropna()
pairs   = list(zip(test_df.question1, test_df.question2))
y_true  = test_df.is_duplicate.values.astype(int)

from sklearn.metrics import log_loss, accuracy_score, f1_score
p = moe.predict_prob(pairs)
print(f"TEST · LL {log_loss(y_true,p):.4f}  ·  ACC {accuracy_score(y_true,p>0.5):.4f}  ·  F1 {f1_score(y_true,p>0.5):.4f}")